## Day 23 - longest path down

**Part 1: Find longest path from single top row point to single bottom row point**

- `>` etc. tiles means you have to go that direction no other choice (stepe slope)

In [3]:
with open("./example.txt") as f:
    example_grid = [[*line.strip()] for line in f.readlines()]

with open("./input.txt") as f:
    input_grid = [[*line.strip()] for line in f.readlines()]

In [4]:
def update_been_there_dict(been_there: dict[tuple, set], old_id: int, new_id: int, base_path_new_to_ignore: tuple[int, int]):
    for k, v in been_there.items():
        if k == base_path_new_to_ignore:
            continue
        if old_id in v:
            been_there[k].add(new_id)

In [5]:
len(example_grid[0])

23

In [6]:
def part1(grid: list[list[str]]) -> int:
    """
    Finding longest possible path from top to bottom.
    """
    # using (y,x) coords with top left 0,0 s.t. access grid[y][x]
    sy, sx = (0, grid[0].index("."))
    ending_pos = (len(grid) - 1, grid[-1].index("."))
    # both input and example have these in (0, 1) and (-1, -2) but this generalises I suppose!

    # (steps, y, x, pathID)
    # initially no steps, at the starting position
    initial_pathID = 1
    # initial = (1e10, sy, sx, initial_pathID)
    initial = (-1, sy, sx, initial_pathID)

    paths = [initial]
    been_there = {
        (sy, sx): {initial_pathID}
    }

    answers = set()
    used_IDs = {1}

    while paths:
        steps, y, x, path_id = paths.pop()

        steps += 1

        if (y, x) == ending_pos:
            answers.add(abs(steps))

        if grid[y][x] == ".":
            # any valid direction
            first = True
            base_path_id = path_id
            for dy, dx in ((1, 0), (-1, 0), (0, 1), (0, -1)):
                ny = y + dy
                nx = x + dx
                if (0 <= ny <= len(grid) - 1) and (0 <= nx <= len(grid[0]) - 1) and path_id not in (been_value := been_there.get((ny, nx), set())):
                    if grid[ny][nx] != "#":
                        new_path_id = path_id
                        if first:
                            first = False
                            base_path_new_to_ignore = (ny, nx)
                        else:
                            # messy way of getting new paths onto new unique ids
                            while True:
                                new_path_id = new_path_id + 1
                                if new_path_id not in used_IDs:
                                    used_IDs.add(new_path_id)
                                    break
                            update_been_there_dict(been_there, base_path_id, new_path_id, base_path_new_to_ignore)
                        # we move
                        paths.append((steps, ny, nx, new_path_id))
                        been_value.add(new_path_id)
                        been_there[(ny, nx)] = been_value
            continue

        if grid[y][x] == ">":
            # go right
            ny = y + 0
            nx = x + 1
        elif grid[y][x] == "v":
            # go down
            ny = y + 1
            nx = x + 0
        elif grid[y][x] == "<":
            # go left  -  also never happens in my input
            ny = y + 0
            nx = x - 1
        elif grid[y][x] == "^":
            # go up  -  never happens in my input
            ny = y - 1
            nx = x + 0
        else:
            raise RuntimeError("Invalid tile!")
        
        if grid[ny][nx] != "#" and path_id not in (been_value := been_there.get((ny, nx), set())):
            paths.append((steps, ny, nx, path_id))
            been_value.add(path_id)
            been_there[(ny, nx)] = been_value

    return max(answers)

assert part1(example_grid) == 94
part1(input_grid)  # a little slow but we're ok with it

2202

**Part 2: The same without steep slopes**

In [17]:
import sys
from copy import deepcopy
def part2(grid: list[list[str]]) -> int:
    """
    Finding longest possible path from top to bottom. But now it's more complicated bc no steep sloped (higher complexity!)
    """
    # using (y,x) coords with top left 0,0 s.t. access grid[y][x]
    sy, sx = (0, grid[0].index("."))
    ending_pos = (len(grid) - 1, grid[-1].index("."))

    # (steps, y, x, pathID)
    # initially no steps, at the starting position
    # initial = (1e10, sy, sx, SEEN)
    initial = (-1, sy, sx, set((sy, sx)))

    paths = [initial]

    answer = 0

    while paths:
        current = paths.pop()
        current_path = [current]
        while current_path:
            steps, y, x, seen = current_path.pop()
            steps += 1
            assert isinstance(seen, set)

            if (y, x) == ending_pos:
                prev_print = answer
                answer = max(answer, steps)
                if answer != prev_print:
                    print(f"New one! {answer=}")
                    print("paths size", sys.getsizeof(paths))
                current_path = []
                break

            # any valid direction
            first = True
            for dy, dx in ((1, 0), (-1, 0), (0, 1), (0, -1)):
                ny = y + dy
                nx = x + dx
                if (0 <= ny <= len(grid) - 1) and (0 <= nx <= len(grid[0]) - 1) and (ny, nx) not in seen:
                    if grid[ny][nx] != "#":
                        if first:
                            seen.add((ny, nx))
                            current_path.append(
                                (steps, ny, nx, seen)
                            )
                            first = False
                        else:
                            new_seen = deepcopy(seen)
                            seen.add((ny, nx))
                            # we move
                            paths.append((steps, ny, nx, new_seen))

    return answer

assert part2(example_grid) == 154
part2(input_grid) # 6226 was correct ! After 900 minutes you too could maybe get an answer !

New one! answer=154
paths size 184
New one! answer=5102
paths size 472
New one! answer=5294
paths size 472
New one! answer=5494
paths size 472
New one! answer=5522
paths size 472
New one! answer=5602
paths size 472
New one! answer=5762
paths size 472
New one! answer=5842
paths size 472
New one! answer=5990
paths size 472
New one! answer=6014
paths size 472
New one! answer=6034
paths size 472
New one! answer=6102
paths size 472
New one! answer=6146
paths size 472
New one! answer=6162
paths size 472
New one! answer=6166
paths size 472
New one! answer=6222
paths size 472
New one! answer=6226
paths size 408


KeyboardInterrupt: 

Well after 900 minutes or so the brute force prevailed by printing the right answer (despite not actually finishing running!)... very amusing

In [38]:
# Here now I've somehow got to the answer sort of legitimately, I'm going to try and implement HyperNeutrino's graph theory solution alongside the video :)

def hyperneutrino_part2(grid: list[list[str]]) -> int:
    # nothing too fancy, but uses edge contraction so just connect the relevant points (as we have a known distance between the points where a 'decision' is made)
    # and then similarly brute force it, but it's far less complicated

    starting_pos = (0, grid[0].index("."))
    ending_pos = (len(grid) - 1, grid[-1].index("."))

    points_of_interest = [starting_pos, ending_pos]

    for r, row in enumerate(grid):
        for c, ch in enumerate(row):
            if ch == "#":
                continue
            neighbours = 0
            # find out the decision points, i.e. where we dont pass straight through and can make a decision
            for nr, nc in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
                if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and grid[nr][nc] != "#":
                    neighbours += 1
                
            if neighbours >= 3:
                points_of_interest.append((r, c))
    
    # build our graph, noting which points are connected to which and the distance between them
    graph = {pt: {} for pt in points_of_interest}

    for sr, sc in points_of_interest:
        stack = [(0, sr, sc)]
        seen = {(sr, sc)}

        while stack:
            n, r, c = stack.pop()
            
            if (
                n != 0  # starting posiition, dont want an edge to itself
                and (r, c) in points_of_interest
            ):
                graph[(sr, sc)][(r, c)] = n  # says graph has a distance from (sr, sc) to (r, c) of n
                continue

            for dr, dc in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
                nr = r + dr
                nc = c + dc
                if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and grid[nr][nc] != "#" and (nr, nc) not in seen:
                    stack.append((n + 1, nr, nc))
                    seen.add((r, c))
                    # essentially we're incrememnting here until we reach a point of interest then that is added and we move on

    seen = set()
    def differences(point: tuple[int, int]):
        if point == ending_pos:
            return 0  # travel length 0 to the end
        
        path_length = -float("inf")  # if none of them find the end point we get -inf

        seen.add(point)
        for next_point in graph[point]:
            if next_point not in seen:
                # avoid cycles by not going back to seen
                path_length = max(path_length, differences(next_point) + graph[point][next_point])
                # keep trying to find distance to end and kind of sum the distance to the next point as we go, but always keeping the biggest distance we can travel each time
        seen.remove(point)

        return path_length

    return differences(starting_pos)



assert hyperneutrino_part2(example_grid) == 154
hyperneutrino_part2(input_grid)

6226

Pretty cool. 900 minutes down to a half a minute! Think I understand it but would've struggled to come up with that recursion myself. But the edge contraction was just sensible I should've done something like that then my own path finding solution.